# Bayesian LSTM for Anomaly Detection in TimeSeries

In [1]:
from pandas import read_csv
from matplotlib import pyplot as plt
import tensorflow.compat.v1 as tf
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
import io

tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


## Dataset Preprocessing

We upload the dataset and extract the Receiving Tank and Heating Tank temperature variables. We compute the moving average of the points to reduce the dimension without losing the overall trend of the data. As our time series present a pattern, we chose to work on 2,000 points. We also sphere the data using sklearn to speed up the training phase.


In [2]:
dataset = read_csv('train_1500000_seed_11_vars_23.csv')

In [3]:
RT = dataset[['Time','RT_temperature.T']].copy()
RT.set_index('Time',inplace=True)
RT = RT.values

HT = dataset[['Time','HT_temperature.T']].copy()
HT.set_index('Time',inplace=True)
HT = HT.values

In [4]:
RT = np.mean(RT.reshape(-1,39),1)
RT = RT[0:2000].reshape(-1,1)
scaler = StandardScaler()
RT_scaled = scaler.fit_transform(RT)

HT = np.mean(HT.reshape(-1,39),1)
HT = HT[0:2000].reshape(-1,1)
scaler = StandardScaler()
HT_scaled = scaler.fit_transform(HT)

Chose which variable you want to predict.

In [5]:
scaled = HT_scaled

n,d = scaled.shape

## Data Partitioning and Parameters Definition

We choose to split the data into 80% training. 
The function window allows us to reshape the data in function of the window size we chose.

In [6]:
def window(data,window_size):
    X = []
    y = []
    
    i=0
    while (i + window_size) <= len(data) - 1:
        X.append(data[i:i+window_size])
        y.append(data[i+window_size])
        
        i+=1
    return X,y

In [7]:
batch_size = 128
window_size = 20
hidden_layer = 50
learning_rate = 0.01
n_params = 4*(2*hidden_layer + hidden_layer**2) + hidden_layer + 1


elbo_loss = []
epoch_counter = 0

In [8]:
ntrain = n - int(0.2*scaled.shape[0])
X, y = window(scaled, window_size)

X_train = np.array(X[:ntrain])
y_train = np.array(y[:ntrain])
t_train = np.arange(X_train.shape[0])

X_test = np.array(X[ntrain:])
y_test = np.array(y[ntrain:])
t_test = np.arange(X_test.shape[0])


print("X_train shape: " + str(X_train.shape))
print("y_train shape: " + str(y_train.shape))
print("X_test shape: " + str(X_test.shape))
print("y_test shape: " + str(y_test.shape))

X_train shape: (1600, 20, 1)
y_train shape: (1600, 1)
X_test shape: (380, 20, 1)
y_test shape: (380, 1)


## Variables Definition

In [9]:
session = tf.InteractiveSession()
particle_tensor = tf.Variable(tf.truncated_normal([1,n_params], stddev=0.05)) 

finish = 0
# Weights for the input gate
start = finish
finish = start + hidden_layer
Wi_gate = tf.reshape(particle_tensor[:, start:finish], (1, 1, hidden_layer))
start = finish
finish = start + hidden_layer ** 2
Wi_hidden = tf.reshape(particle_tensor[:, start: finish], (1, hidden_layer, hidden_layer))
start = finish
finish = start + hidden_layer
bi = tf.reshape(particle_tensor[:, start: finish], (1, 1, hidden_layer))

#Weights for the forget gate
start = finish
finish = start + hidden_layer
Wf_gate = tf.reshape(particle_tensor[:, start:finish], (1, 1, hidden_layer))
start = finish
finish = start + hidden_layer ** 2
Wf_hidden = tf.reshape(particle_tensor[:, start: finish], (1, hidden_layer, hidden_layer))
start = finish
finish = start + hidden_layer
bf = tf.reshape(particle_tensor[:, start: finish], (1, 1, hidden_layer))
bf_biased = tf.reshape(tf.Variable(3*tf.ones([hidden_layer])), (1, 1, hidden_layer))

#Weights for the output gate
start = finish
finish = start + hidden_layer
Wo_gate = tf.reshape(particle_tensor[:, start:finish], (1, 1, hidden_layer))
start = finish
finish = start + hidden_layer ** 2
Wo_hidden = tf.reshape(particle_tensor[:, start: finish], (1, hidden_layer, hidden_layer))
start = finish
finish = start + hidden_layer
bo = tf.reshape(particle_tensor[:, start: finish], (1, 1, hidden_layer))

#weights for the memory cell
start = finish
finish = start + hidden_layer
Wc_gate = tf.reshape(particle_tensor[:, start:finish], (1, 1, hidden_layer))
start = finish
finish = start + hidden_layer ** 2
Wc_hidden = tf.reshape(particle_tensor[:, start: finish], (1, hidden_layer, hidden_layer))
start = finish
finish = start + hidden_layer
bc = tf.reshape(particle_tensor[:, start: finish], (1, 1, hidden_layer))

#Output layer weights
start = finish
finish = start + hidden_layer
weights_output = tf.reshape(particle_tensor[:, start:finish], (1, hidden_layer, 1))
start = finish
finish = start + 1
bias_output = tf.reshape(particle_tensor[:, start: finish], (1, 1, 1))
start = finish

## Definition of the different neural networks cell

In [10]:
def LSTM_cell(input,output,state, small_dim=False):

    if small_dim:
        input_gate =  tf.sigmoid(tf.einsum('mij,njk->mnik', input, Wi_gate) 
                                 + tf.einsum('ij,njk->nik', output, Wi_hidden) + bi)
        
        forget_gate =  tf.sigmoid(tf.einsum('mij,njk->mnik', input, Wf_gate) 
                                  + tf.einsum('ij,njk->nik', output, Wf_hidden) + bf)

        output_gate =  tf.sigmoid(tf.einsum('mij,njk->mnik', input, Wo_gate) 
                                  + tf.einsum('ij,njk->nik', output, Wo_hidden) +bo)

        memory_cell =  tf.tanh(tf.einsum('mij,njk->mnik', input, Wc_gate) 
                               + tf.einsum('ij,njk->nik', output, Wc_hidden) + bc)
        
        state = tf.reshape(state, [-1, 1, hidden_layer]) * forget_gate + input_gate * memory_cell

        
    else:
        input_gate =  tf.sigmoid(tf.einsum('mij,njk->mnik', input, Wi_gate) 
                                 + tf.einsum('...nij,njk->...nik', output, Wi_hidden) + bi)
        
        forget_gate =  tf.sigmoid(tf.einsum('mij,njk->mnik', input, Wf_gate) 
                                  + tf.einsum('...nij,njk->...nik', output, Wf_hidden) + bf)

        output_gate =  tf.sigmoid(tf.einsum('mij,njk->mnik', input, Wo_gate) 
                                  + tf.einsum('...nij,njk->...nik', output, Wo_hidden) +bo)

        memory_cell =  tf.tanh(tf.einsum('mij,njk->mnik', input, Wc_gate) 
                               + tf.einsum('...nij,njk->...nik', output, Wc_hidden) + bc)
        
        state = tf.reshape(state, [-1, 1, 1, hidden_layer]) * forget_gate + input_gate * memory_cell

    
    output = output_gate * tf.tanh(state)
    
    return state, output

In [11]:
def LSTM_cell_peephole(input,output,state, small_dim=False):
    if small_dim:
        input_gate =  tf.sigmoid(tf.einsum('mij,njk->mnik', input, Wi_gate) 
                                 + tf.einsum('ij,njk->nik', output, Wi_hidden) + bi)
        
        forget_gate =  tf.sigmoid(tf.einsum('mij,njk->mnik', input, Wf_gate) 
                                  + tf.einsum('ij,njk->nik', output, Wf_hidden) + bf)

        output_gate =  tf.sigmoid(tf.einsum('mij,njk->mnik', input, Wo_gate) 
                                  + tf.einsum('ij,njk->nik', output, Wo_hidden) +bo)

        memory_cell =  tf.tanh(tf.einsum('mij,njk->mnik', input, Wc_gate) + bc)
        
        state = tf.reshape(state, [-1, 1, hidden_layer]) * forget_gate + input_gate * memory_cell

        
    else:
        input_gate =  tf.sigmoid(tf.einsum('mij,njk->mnik', input, Wi_gate) 
                                 + tf.einsum('...nij,njk->...nik', output, Wi_hidden) + bi)
        
        forget_gate =  tf.sigmoid(tf.einsum('mij,njk->mnik', input, Wf_gate) 
                                  + tf.einsum('...nij,njk->...nik', output, Wf_hidden) + bf)

        output_gate =  tf.sigmoid(tf.einsum('mij,njk->mnik', input, Wo_gate) 
                                  + tf.einsum('...nij,njk->...nik', output, Wo_hidden) +bo)

        memory_cell =  tf.tanh(tf.einsum('mij,njk->mnik', input, Wc_gate) + bc)
        
        state = tf.reshape(state, [-1, 1, 1, hidden_layer]) * forget_gate + input_gate * memory_cell

    
    output = tf.tanh(output_gate * state)
    
    return state, output

In [12]:
def LSTM_cell_biased(input,output,state, small_dim=False):

    if small_dim:
        input_gate =  tf.sigmoid(tf.einsum('mij,njk->mnik', input, Wi_gate) 
                                 + tf.einsum('ij,njk->nik', output, Wi_hidden) + bi)
        
        forget_gate =  tf.sigmoid(tf.einsum('mij,njk->mnik', input, Wf_gate) 
                                  + tf.einsum('ij,njk->nik', output, Wf_hidden) + bf_biased)

        output_gate =  tf.sigmoid(tf.einsum('mij,njk->mnik', input, Wo_gate) 
                                  + tf.einsum('ij,njk->nik', output, Wo_hidden) +bo)

        memory_cell =  tf.tanh(tf.einsum('mij,njk->mnik', input, Wc_gate) 
                               + tf.einsum('ij,njk->nik', output, Wc_hidden) + bc)
        
        state = tf.reshape(state, [-1, 1, hidden_layer]) * forget_gate + input_gate * memory_cell

        
    else:
        input_gate =  tf.sigmoid(tf.einsum('mij,njk->mnik', input, Wi_gate) 
                                 + tf.einsum('...nij,njk->...nik', output, Wi_hidden) + bi)
        
        forget_gate =  tf.sigmoid(tf.einsum('mij,njk->mnik', input, Wf_gate) 
                                  + tf.einsum('...nij,njk->...nik', output, Wf_hidden) + bf_biased)

        output_gate =  tf.sigmoid(tf.einsum('mij,njk->mnik', input, Wo_gate) 
                                  + tf.einsum('...nij,njk->...nik', output, Wo_hidden) +bo)

        memory_cell =  tf.tanh(tf.einsum('mij,njk->mnik', input, Wc_gate) 
                               + tf.einsum('...nij,njk->...nik', output, Wc_hidden) + bc)
        
        state = tf.reshape(state, [-1, 1, 1, hidden_layer]) * forget_gate + input_gate * memory_cell

    
    output = output_gate * tf.tanh(state)
    
    return state, output

In [13]:
def RNN_cell(input,output, small_dim=False):
    if small_dim:
        output_gate =  tf.sigmoid(tf.einsum('...ij,njk->...nik', input, Wo_gate) 
                                  + tf.einsum('ij,njk->nik', output, Wo_hidden) +bo)
    else:
        output_gate =  tf.sigmoid(tf.einsum('...ij,njk->...nik', input, Wo_gate) 
                                  + tf.einsum('...nij,njk->...nik', output, Wo_hidden) +bo)

    output = output_gate
    
    return output

## Definition of the output

In [14]:
def output(window_size, architecture):
    if architecture == RNN_cell:
        inputs = tf.placeholder(tf.float32, [None,window_size,1])
        targets = tf.placeholder(tf.float32, [None, 1])

        batch_state0 = tf.constant(np.zeros([1, hidden_layer], dtype = np.float32))
        batch_output0 = tf.constant(np.zeros([1, hidden_layer], dtype = np.float32))

        for k in range(window_size):
            if k == 0:
                batch_output = architecture(tf.reshape(inputs[:, k], (-1,1,1)), batch_state0, True)
            else:
                batch_output = architecture(tf.reshape(inputs[:, k], (-1,1,1)), batch_output)

        outputs = tf.matmul(batch_output, weights_output) + bias_output
        diff_tensor = tf.reshape(outputs - tf.reshape(targets, (-1, 1, 1, 1)), (-1, 1, 1))
        mse = tf.reduce_mean(diff_tensor ** 2)
        
    else:
        inputs = tf.placeholder(tf.float32, [None,window_size,1])
        targets = tf.placeholder(tf.float32, [None, 1])

        batch_state0 = tf.constant(np.zeros([1, hidden_layer], dtype = np.float32))
        batch_output0 = tf.constant(np.zeros([1, hidden_layer], dtype = np.float32))

        #Choose which architecture to use
        for k in range(window_size):
            if k == 0:
                batch_state, batch_output = architecture(tf.reshape(inputs[:, k], (-1,1,1)), batch_state0, batch_output0, True)
            else:
                batch_state, batch_output = architecture(tf.reshape(inputs[:, k], (-1,1,1)), batch_state, batch_output)

        outputs = tf.matmul(batch_output, weights_output) + bias_output

        diff_tensor = tf.reshape(outputs - tf.reshape(targets, (-1, 1, 1, 1)), (-1, 1, 1))
        mse = tf.reduce_mean(diff_tensor ** 2)
        
    return targets, inputs, outputs, mse

Chose the architecture you want to use

In [15]:
architecture = RNN_cell

## Optimizer setup

In [16]:
targets, inputs, outputs, mse = output(window_size, architecture)
optimizer = tf.train.AdamOptimizer(learning_rate)
step = optimizer.minimize(mse)
session.run(tf.global_variables_initializer())

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


## Training

In [17]:
def train(epochs, epoch_counter):
    print('Training...')
    max_epochs = epoch_counter + epochs
    for i in range(epochs):
        epoch_counter += 1
        ii = 0
        order = np.arange(ntrain)
        np.random.seed(i)
        np.random.shuffle(order)
        while (ii+batch_size) <= X_train.shape[0]:
            epoch_loss = []
            X_batch = X_train[order][ii:ii+batch_size]
            y_batch = y_train[order][ii:ii+batch_size]


            session.run(step, feed_dict = {inputs:X_batch, targets: y_batch})
            o, c = session.run([outputs, mse], feed_dict = {inputs:X_batch, targets: y_batch})

            epoch_loss.append(c)

            ii+= batch_size

        elbo_loss.append(np.mean(epoch_loss))
        print('Epoch {}/{}'.format(epoch_counter, max_epochs), ' Current loss: {}'.format(elbo_loss[-1]))

In [18]:
if architecture == RNN_cell:
    epochs = 20
else:
    epochs = 100
train(epochs, len(elbo_loss))

Training...
Epoch 1/20  Current loss: 0.7669117450714111
Epoch 2/20  Current loss: 0.5388635993003845
Epoch 3/20  Current loss: 0.10538101941347122
Epoch 4/20  Current loss: 0.08163900673389435
Epoch 5/20  Current loss: 0.07637578248977661
Epoch 6/20  Current loss: 0.033758051693439484
Epoch 7/20  Current loss: 0.06191442906856537
Epoch 8/20  Current loss: 0.04613608121871948
Epoch 9/20  Current loss: 0.029628220945596695
Epoch 10/20  Current loss: 0.030433237552642822
Epoch 11/20  Current loss: 0.02373000979423523
Epoch 12/20  Current loss: 0.04943377152085304
Epoch 13/20  Current loss: 0.06760505586862564
Epoch 14/20  Current loss: 0.03330322355031967
Epoch 15/20  Current loss: 0.0176177266985178
Epoch 16/20  Current loss: 0.032783277332782745
Epoch 17/20  Current loss: 0.044800229370594025
Epoch 18/20  Current loss: 0.03397064283490181
Epoch 19/20  Current loss: 0.04631730914115906
Epoch 20/20  Current loss: 0.02042626217007637


## Predict

In [19]:
y_predict = session.run(outputs, feed_dict = {inputs:X_test, targets: y_test}).squeeze()

## Reverse the sphering

In [20]:
y_predict_scaled = scaler.inverse_transform(y_predict).flatten()
y_test_scaled = scaler.inverse_transform(y_test).flatten()

## Calculate the MSE

In [21]:
MSE = np.square(y_predict_scaled[:128] - y_test_scaled[:128]).mean()

In [22]:
print('The MSE is: ' + str(MSE))

The MSE is: 4.054698290394015
